# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices. **This Series should be sorted by a tie-break sort in the format of ("extracted date", "original row number").**

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [500]:
import pandas as pd

doc = []
with open('assets/dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [629]:
def date_sorter():
    # doc = []
    # with open('assets/dates.txt') as file:
    #     for line in file:
    #         doc.append(line)

    df = pd.Series(doc)
    df = df.to_frame('Text')
    df['Text'] = df['Text'].str.replace('Janaury', 'January')
    df['Text'] = df['Text'].str.replace('Decemeber', 'December')
    df['extracted_date'] = pd.NA
    #  date pattern 04/20/2009; 04/20/09; 4/20/09; 4/3/09
    # Regular expression pattern
    date_patterns = [   # Standard Dates (mm/dd/yyyy and variations)
                                    r'((?:0?[1-9]|1[0-2])\/(?:0?[1-9]|[12]\d|3[01])\/(?:\d{2}|\d{4}))\b',
                                    r'\b(\d{1,2}-\d{1,2}-\d{2})\b',
                                    # Month Name, Day, Year (with variations in month name length and separators)
                                    r'\b((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\.?\s\d{1,2}[,\s-]?\s\d{4})\b',
                                    r'\b(\d{1,2}\s(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\.?,?\s\d{4})\b',
                                    r'\b((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s\d{1,2}(?:th|st|nd|rd),\s\d{4})\b',
                                    r'(\d{1,2}/\d{4})',

                                    # Month Name and Year (with variations in month name length)
                                    r'\b((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\.?\s\d{4})\b',
                                    r'\b((?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4})\b',
                                    r'\b.((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\s?\d{4})\b',
                                    r'\b((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*,\s\d{4})\b',
                                    # Month/Year
                                    r'\b(\d{1,2}/\d{4})\b',
                                    # Year Only
                                    r'\b(?:[^\d]*)(\d{4})\b'
] 

    # Extract dates if column exctraced text is Nan
    # Apply str.extract only where 'extracted_date' is NaN
    for i in date_patterns:
        mask = df['extracted_date'].isna()
        extracted_data = df.loc[mask, 'Text'].str.extract(i)[0]
        df.loc[mask, 'extracted_date'] = extracted_data.where(~extracted_data.isna(), df.loc[mask, 'extracted_date'])

    #manual handling
    df.at[271,'extracted_date'] = 'August 2008'
    df.at[321,'extracted_date'] = '2 June 1999'
    df.at[274,'extracted_date'] = '7 April 1985'
    df['transformed_date'] = pd.to_datetime(df['extracted_date'])

    # handling of transforming to datetype with two digit years
    pattern = r'\b(?:1[0-2]|0?[1-9])\/(?:3[01]|[12][0-9]|0?[1-9])\/\d{2}\b'
    df['is_valid_date'] = df['extracted_date'].str.match(pattern)
    df['correct_year'] = df[(df['is_valid_date']) & (df['transformed_date'].dt.year > 2000)]['transformed_date'].apply(lambda x: x - pd.DateOffset(years=100))

    # Update 'transformed_date' where 'correct_year' is not NaT
    mask = pd.notna(df['correct_year'])
    df.loc[mask, 'transformed_date'] = df.loc[mask, 'correct_year']
    df.reset_index(inplace=True)    

    df.sort_values(by=['transformed_date','index'],inplace=True)
    df['Count'] = range(len(df))
    df.set_index('Count',inplace=True)
    series = df['index']
    return series



In [631]:
print(date_sorter().head(30))

Count
0       9
1      84
2       2
3      53
4      28
5     474
6     153
7      13
8     129
9      98
10    111
11    225
12     31
13    171
14    191
15    486
16    335
17    415
18     36
19    323
20    405
21    422
22    375
23    380
24    345
25     57
26    481
27    436
28    104
29    299
Name: index, dtype: int64


/tmp/ipykernel_18533/205870813.py:42: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['transformed_date'] = pd.to_datetime(df['extracted_date'])


In [636]:
doc = []
with open('assets/dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df = df.to_frame('Text')
df['Text'] = df['Text'].str.replace('Janaury', 'January')
df['Text'] = df['Text'].str.replace('Decemeber', 'December')
df['extracted_date'] = pd.NA
#  date pattern 04/20/2009; 04/20/09; 4/20/09; 4/3/09
# Regular expression pattern
date_patterns = [   # Standard Dates (mm/dd/yyyy and variations)
                                    r'((?:0?[1-9]|1[0-2])\/(?:0?[1-9]|[12]\d|3[01])\/(?:\d{2}|\d{4}))\b',
                                    r'\b(\d{1,2}-\d{1,2}-\d{2})\b',
                                    # Month Name, Day, Year (with variations in month name length and separators)
                                    r'\b((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\.?\s\d{1,2}[,\s-]?\s\d{4})\b',
                                    r'\b(\d{1,2}\s(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\.?,?\s\d{4})\b',
                                    r'\b((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s\d{1,2}(?:th|st|nd|rd),\s\d{4})\b',
                                    r'(\d{1,2}/\d{4})',

                                    # Month Name and Year (with variations in month name length)
                                    r'\b((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\.?\s\d{4})\b',
                                    r'\b((?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4})\b',
                                    r'\b.((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\s?\d{4})\b',
                                    r'\b((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*,\s\d{4})\b',
                                    # Month/Year
                                    r'\b(\d{1,2}/\d{4})\b',
                                    # Year Only
                                    r'\b(?:[^\d]*)(\d{4})\b'
]       
# Extract dates if column exctraced text is Nan
# Apply str.extract only where 'extracted_date' is NaN
for i in date_patterns:
    mask = df['extracted_date'].isna()
    extracted_data = df.loc[mask, 'Text'].str.extract(i)[0]
    df.loc[mask, 'extracted_date'] = extracted_data.where(~extracted_data.isna(), df.loc[mask, 'extracted_date'])

#manual handling
df.at[271,'extracted_date'] = 'August 2008'
df.at[321,'extracted_date'] = '2 June 1999'
df.at[274,'extracted_date'] = '7 April 1985'
df['transformed_date'] = pd.to_datetime(df['extracted_date'])

# handling of transforming to datetype with two digit years
pattern = r'\b(?:1[0-2]|0?[1-9])\/(?:3[01]|[12][0-9]|0?[1-9])\/\d{2}\b'
df['is_valid_date'] = df['extracted_date'].str.match(pattern)
df['correct_year'] = df[(df['is_valid_date']) & (df['transformed_date'].dt.year > 2000)]['transformed_date'].apply(lambda x: x - pd.DateOffset(years=100))

# Update 'transformed_date' where 'correct_year' is not NaT
mask = pd.notna(df['correct_year'])
df.loc[mask, 'transformed_date'] = df.loc[mask, 'correct_year']
df.reset_index(inplace=True)    

df.sort_values(by=['transformed_date','index'],inplace=True)
df['Count'] = range(len(df))
df.set_index('Count',inplace=True)
series = df['index']

/tmp/ipykernel_18533/1624289103.py:43: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['transformed_date'] = pd.to_datetime(df['extracted_date'])


In [632]:
df.to_csv('test1.csv')

In [633]:
df.iloc[414]

index                               173
Text                 18 Oct 2006 Age:\n
extracted_date              18 Oct 2006
transformed_date    2006-10-18 00:00:00
is_valid_date                     False
correct_year                        NaT
Name: 414, dtype: object

In [637]:
a = df[df['index'] == 414]
print(a)

       index                                         Text extracted_date  \
Count                                                                      
390      414  n4/2004 R mastectomy with reconstruction.\n         4/2004   

      transformed_date  is_valid_date correct_year  
Count                                               
390         2004-04-01          False          NaT  


In [607]:
pattern = r'\b(\d{1,2})(January|February|March|April|May|June|July|August|September|October|November|December)[,]?\s(\d{4})\b'

extracted_dates11 = df['Text'].str.extract(pattern)

df['test1'] = df['Text'].str.extract(pattern)[0]
df['test2'] = df['Text'].str.extract(pattern)[1]
df['test3'] = df['Text'].str.extract(pattern)[2]




In [606]:
df.iloc[339]

index                                                321
Text                2June, 1999 Audit C Score Current:\n
extracted_date                                      1999
transformed_date                     1999-01-01 00:00:00
is_valid_date                                      False
correct_year                                         NaT
test1                                                  2
test2                                               June
test3                                               1999
Name: 339, dtype: object

In [611]:
df[df['index'] == 321]

,index,Text,extracted_date,transformed_date,is_valid_date,correct_year,test1,test2,test3,formatted_date
Count,,,,,,,,,,
339,321,"2June, 1999 Audit C Score Current:\n",1999,1999-01-01,False,NaT,2,June,1999,2 June 1999


In [610]:
df[df['index'] == 311]

,index,Text,extracted_date,transformed_date,is_valid_date,correct_year,test1,test2,test3,formatted_date
Count,,,,,,,,,,
295,311,"- Prozac 20 mg daily: February, 1995: self-di...","February, 1995",1995-02-01,False,NaT,NaN,NaN,NaN,NaN


In [608]:
df['formatted_date'] = extracted_dates11[0] + ' ' + extracted_dates11[1] + ' ' + extracted_dates11[2]


In [609]:
df.iloc[]

,index,Text,extracted_date,transformed_date,is_valid_date,correct_year,test1,test2,test3,formatted_date
Count,,,,,,,,,,
0,9,(4/10/71)Score-1Audit C Score Current:\n,4/10/71,1971-04-10,True,1971-04-10,NaN,NaN,NaN,NaN
1,84,5/18/71 Total time of visit (in minutes):\n,5/18/71,1971-05-18,True,1971-05-18,NaN,NaN,NaN,NaN
2,2,sshe plans to move as of 7/8/71 In-Home Servic...,7/8/71,1971-07-08,True,1971-07-08,NaN,NaN,NaN,NaN
3,53,7/11/71 SOS-10 Total Score:\n,7/11/71,1971-07-11,True,1971-07-11,NaN,NaN,NaN,NaN
4,28,9/12/71 [report_end]\n,9/12/71,1971-09-12,True,1971-09-12,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
495,427,6e. monitor pt's depressive experience in cont...,5/2016,2016-05-01,False,NaT,NaN,NaN,NaN,NaN
496,141,30 May 2016 SOS-10 Total Score:\n,30 May 2016,2016-05-30,False,NaT,NaN,NaN,NaN,NaN
497,186,13 Oct 2016 Primary Care Doctor:\n,13 Oct 2016,2016-10-13,False,NaT,NaN,NaN,NaN,NaN
